In [1]:
%load_ext tensorboard
import os
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
window_size = 25

def cubic_sentences(file_dir):
    phoneme_sentences = []
    for filename in file_dir:
        df = np.array(pd.read_csv(filename, header=None), dtype='float32')

        check = 0
        tmp = []
        for i in range(df.shape[0]):
            if df[i,1] == 10000:
                if check == 0:
                    data_per_window = tmp
                    tmp = []
                    check = 1
                else:
                    data_per_window = np.dstack((data_per_window,tmp))
                    tmp = []
            else:
                if len(tmp) == 0:
                    tmp = df[i,:]
                else:            
                    tmp = np.vstack((tmp,df[i,:]))
        
        data_per_window = np.moveaxis(data_per_window, -1, 0)
        phoneme_sentences.append(data_per_window)
        
    return phoneme_sentences # list타입, 각 element에는 cube형태의 sentence샘플, sentence샘플의 shape: (sentence_length, window_size, feature_dim)

def normalize_sentences(left_sentences, right_sentences):
    sentences = left_sentences + right_sentences
    count = 0
    for i, stc in enumerate(sentences):
        for j in range(stc.shape[0]):
            each_timestep = stc[j,:,:]
            if count==0:
                stacked_sentences = each_timestep
            else:
                stacked_sentences = np.vstack((stacked_sentences, each_timestep))
            
            count = count + 1
        
    sentence_mean = np.mean(stacked_sentences, axis=0) # (1,feature_dim)
    sentence_std = np.std(stacked_sentences, axis=0) # (1,feature_dim)
    for i, stc in enumerate(sentences):
        sentences[i] = (stc - sentence_mean)/sentence_std
        
    output_left_sentences = sentences[:len(left_sentences)]
    output_right_sentences = sentences[len(left_sentences):]
        
    return output_left_sentences, output_right_sentences, sentence_mean, sentence_std

def normalize_dict(phoneme_dict, feature_mean, feature_std):
    output_phoneme_dict = (phoneme_dict - feature_mean)/feature_std
    return output_phoneme_dict
    
def zero_padding_sentences(sentences,max_stc_len):
    for i in range(len(sentences)):
        tmp = sentences[i]
        padded = np.vstack((tmp,np.zeros((max_stc_len-len(tmp),tmp.shape[1],tmp.shape[2]))))
        sentences[i] = padded
    
    return sentences

def convert_to_4D(sentences,max_stc_len):
    dataset = np.empty((len(sentences),max_stc_len,sentences[0].shape[1],sentences[0].shape[2]))
    for i in range(dataset.shape[0]):
        dataset[i,:,:,:] = sentences[i]
        
    return dataset

# Get list of all the sentences
HSL_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\HSL"
HSR_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\HSR"
LL_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\LL"
LR_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\LR"
ML_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\ML"
MR_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\MR"

HSL_all_files = glob(HSL_path+"\\*.csv")
HSR_all_files = glob(HSR_path+"\\*.csv")
LL_all_files = glob(LL_path+"\\*.csv")
LR_all_files = glob(LR_path+"\\*.csv")
ML_all_files = glob(ML_path+"\\*.csv")
MR_all_files = glob(MR_path+"\\*.csv")

HSL_sentences = cubic_sentences(HSL_all_files)
HSR_sentences = cubic_sentences(HSR_all_files)
LL_sentences = cubic_sentences(LL_all_files)
LR_sentences = cubic_sentences(LR_all_files)
ML_sentences = cubic_sentences(ML_all_files)
MR_sentences = cubic_sentences(MR_all_files)

# HSL_sentences, HSR_sentences, HS_mean, HS_std = normalize_sentences(HSL_sentences, HSR_sentences)
# LL_sentences, LR_sentences, L_mean, L_std = normalize_sentences(LL_sentences, LR_sentences)
# ML_sentences, MR_sentences, M_mean, M_std = normalize_sentences(ML_sentences, MR_sentences)

HS_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\phoneme_dict\\Handshape"
HS_file = glob(HS_path+"\\"+"HS_normalized.csv")
HS_dict = np.array(pd.read_csv(HS_file[0], header=None), dtype='float32')
# HS_dict = normalize_dict(HS_dict, HS_mean, HS_std)
HS_dict = tf.convert_to_tensor(np.transpose(HS_dict))

L_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\phoneme_dict\\Location"
L_file = glob(L_path+"\\"+"L_normalized.csv")
L_dict = np.array(pd.read_csv(L_file[0], header=None), dtype='float32')
# L_dict = normalize_dict(L_dict, L_mean, L_std)
L_dict = tf.convert_to_tensor(np.transpose(L_dict))

M_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\phoneme_dict\\Movement"
M_file = glob(M_path+"\\"+"M_normalized.csv")
M_dict = np.array(pd.read_csv(M_file[0], header=None), dtype='float32')
# M_dict = normalize_dict(M_dict, M_mean, M_std)
M_dict = tf.convert_to_tensor(np.transpose(M_dict))

# 문장 샘플 중 가장 긴 길이 찾기.
# max_stc_len = max([len(stc) for stc in HSL_sentences])
# HSL_sentences = zero_padding_sentences(HSL_sentences,max_stc_len)
# HSR_sentences = zero_padding_sentences(HSR_sentences,max_stc_len)
# LL_sentences = zero_padding_sentences(LL_sentences,max_stc_len)
# LR_sentences = zero_padding_sentences(LR_sentences,max_stc_len)
# ML_sentences = zero_padding_sentences(ML_sentences,max_stc_len)
# MR_sentences = zero_padding_sentences(MR_sentences,max_stc_len)

# 각 phoneme별 sentences를 4차원 numpy array로 변환. (num of sentence sample, timestep, window size, feature dim)
# HSL_dataset = convert_to_4D(HSL_sentences, max_stc_len)
# HSR_dataset = convert_to_4D(HSR_sentences, max_stc_len)
# LL_dataset = convert_to_4D(LL_sentences, max_stc_len)
# LR_dataset = convert_to_4D(LR_sentences, max_stc_len)
# ML_dataset = convert_to_4D(ML_sentences, max_stc_len)
# MR_dataset = convert_to_4D(MR_sentences, max_stc_len)

stc_seq = [[0,1,2],[3,4,5],[6,7,2],[8,5,9],[10,8,11],[4,2,9],[12,13,8,5],
          [0,14,15,16],[0,13,17,18],[19,13,1,20],[3,13,1,5],[15,8,6,21],
          [15,1,22,5],[10,7,0,18],[11,14,23,24,20],[0,7,23,22,5],[15,7,23,24,21],
          [10,8,23,24,21]] # 0부터 인덱싱
num_words = 25
labels = []
for filename in HSL_all_files:
    temp = filename.split(os.path.sep)[-1].split(".csv")[0]
    temp = temp.split('-')[1]
    temp = temp[8:]
    labels.append(np.array(stc_seq[int(temp)-1]))
labels = tf.ragged.constant(labels)
#labels = np.array(labels)
#labels = tf.convert_to_tensor(labels, dtype=tf.int32)

In [3]:
HSL_dataset = tf.ragged.constant(HSL_sentences, inner_shape=(window_size,10))
# HSL_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(HSL_dataset.to_tensor()))
HSR_dataset = tf.ragged.constant(HSR_sentences, inner_shape=(window_size,10))
# HSR_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(HSR_dataset.to_tensor()))
LL_dataset = tf.ragged.constant(LL_sentences, inner_shape=(window_size,12))
# LL_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(LL_dataset.to_tensor()))
LR_dataset = tf.ragged.constant(LR_sentences, inner_shape=(window_size,12))
# LR_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(LR_dataset.to_tensor()))
ML_dataset = tf.ragged.constant(ML_sentences, inner_shape=(1,window_size+3))
# ML_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(ML_dataset.to_tensor()))
MR_dataset = tf.ragged.constant(MR_sentences, inner_shape=(1,window_size+3))
# MR_dataset = tf.RaggedTensor.from_tensor(tf.random.shuffle(MR_dataset.to_tensor()))

indices = tf.range(start=0, limit=len(HSL_sentences))
shuffled_indices = tf.random.shuffle(indices)

HSL_dataset = tf.gather(HSL_dataset, shuffled_indices)
HSR_dataset = tf.gather(HSR_dataset, shuffled_indices)
LL_dataset = tf.gather(LL_dataset, shuffled_indices)
LR_dataset = tf.gather(LR_dataset, shuffled_indices)
ML_dataset = tf.gather(ML_dataset, shuffled_indices)
MR_dataset = tf.gather(MR_dataset, shuffled_indices)
labels = tf.gather(labels, shuffled_indices)

def encode_dict(HSL, HSR, LL, LR, ML, MR, label):
    return{"HSL": HSL, "HSR": HSR, "LL": LL, "LR":LR, "ML": ML, "MR": MR, "sentence": label}

# train_dataset = tf.data.Dataset.from_tensor_slices((HSL_dataset, HSR_dataset, LL_dataset, LR_dataset, ML_dataset, MR_dataset, labels))
# train_dataset = train_dataset.map(encode_dict, num_parallel_calls=tf.data.experimental.AUTOTUNE
#                                  ).shuffle(100).batch(1).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_dataset = encode_dict(HSL_dataset, HSR_dataset, LL_dataset, LR_dataset, ML_dataset, MR_dataset, labels)

In [4]:
# HS_dict = tf.convert_to_tensor(np.random.rand(10,4),dtype='float32')
# L_dict = tf.convert_to_tensor(np.random.rand(12,3),dtype='float32')
# M_dict = tf.convert_to_tensor(np.random.rand(window_size+3,3),dtype='float32')

class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost
        
    def call(self, y_true, y_pred):
#         y_pred = y_pred.to_tensor()
        # compute the training-time loss value and add it
        # to the layer using 'self.add_loss()'
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64") # shape의 [0] 차원은 batch_size를 의미함.
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64") # 매 timestep마다 output 출력하므로 y_pred의 길이가 input의 길이와 동일.      
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64") # 실제 출력되어야 할 문장의 길이. 이 예시 경우엔 5. 
        
        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64") # input_length를 batch_size만큼 복제함.
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64") # label_length를 batch_size만큼 복제함.
        
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)
        
        # At test time, just return the computed predictions
        return y_pred
    
class AttentionLayer(keras.layers.Layer):
    def __init__(self, phoneme_dict):
        super(AttentionLayer, self).__init__()
        self.phoneme_dict = phoneme_dict
#         self.W1 = layers.Dense(units)
#         self.W2 = layers.Dense(units)
#         self.V = layers.Dense(1)
    
    def build(self, input_shape):
        # 학습 가능한 가중치 변수 만들기
        super(AttentionLayer, self).build(input_shape)
        self.kernel = self.add_weight("kernel",
                                     shape=[int(input_shape[-1]),
                                           self.phoneme_dict.shape[0]])

    def call(self, inputs): # HS_dict의 shape: (10,18)=(feature dimension, number of phonemes)
        self.inputs = inputs
        # BahdanauAttention
#         a = self.V(tf.nn.tanh(
#             self.W1(self.phoneme_dict) + self.W2(inputs)))
#         print(a.shape)
        
        # General
        tmp = tf.matmul(self.kernel, self.phoneme_dict)
        score = tf.matmul(inputs,tmp)

        # dot attention 사용 시
#         score = tf.matmul(inputs, self.phoneme_dict)
        split_timestep = tf.split(score, num_or_size_splits=inputs.shape[1], axis=1) # return = list of tensor
        for i in range(len(split_timestep)):
            alpha_each_time = tf.nn.softmax(split_timestep[i])
            context_tmp = tf.matmul(tf.linalg.diag(alpha_each_time), tf.transpose(self.phoneme_dict))
            context_each_time = tf.reduce_sum(context_tmp,2)
            if i==0:
                context_all_time = context_each_time
            else:
                context_all_time = tf.concat([context_all_time, context_each_time],1)
        
        return context_all_time
    
def build_model():
    with tf.device('/gpu:0'):
        # Inputs to the model
        input_HSL = layers.Input(
            shape=(None,window_size,10), name="HSL", dtype="float32", ragged=True)
        
        input_HSR = layers.Input(
            shape=(None,window_size,10), name="HSR", dtype="float32", ragged=True)
        
        input_LL = layers.Input(
            shape=(None,window_size,12), name="LL", dtype="float32", ragged=True)
        
        input_LR = layers.Input(
            shape=(None,window_size,12), name="LR", dtype="float32", ragged=True)        
        
        input_ML = layers.Input(
            shape=(None,1,window_size+3), name="ML", dtype="float32", ragged=True)        
        
        input_MR = layers.Input(
            shape=(None,1,window_size+3), name="MR", dtype="float32", ragged=True)        
        
        labels = layers.Input(shape=(None,), name='sentence', dtype="float32", ragged=True)

        context_mat_HSL = layers.TimeDistributed(AttentionLayer(HS_dict), name="HSLAttention")(input_HSL)
        context_mat_HSR = layers.TimeDistributed(AttentionLayer(HS_dict), name="HSRAttention")(input_HSR)
        context_mat_LL = layers.TimeDistributed(AttentionLayer(L_dict), name="LLAttention")(input_LL)
        context_mat_LR = layers.TimeDistributed(AttentionLayer(L_dict), name="LRAttention")(input_LR)
        context_mat_ML = layers.TimeDistributed(AttentionLayer(M_dict), name="MLAttention")(input_ML)
        context_mat_MR = layers.TimeDistributed(AttentionLayer(M_dict), name="MRAttention")(input_MR)      
        
#         context_mat_HSL = layers.Concatenate(axis=3)([context_mat_HSL,input_HSL])
#         context_mat_HSR = layers.Concatenate(axis=3)([context_mat_HSR,input_HSR])
#         context_mat_LL = layers.Concatenate(axis=3)([context_mat_LL,input_LL])
#         context_mat_LR = layers.Concatenate(axis=3)([context_mat_LR,input_LR])
#         context_mat_ML = layers.Concatenate(axis=3)([context_mat_ML,input_ML])
#         context_mat_MR = layers.Concatenate(axis=3)([context_mat_MR,input_MR])

        h_HSL = layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu"), name="HSLcontext")(context_mat_HSL)
        h_HSL = layers.TimeDistributed(layers.Flatten())(h_HSL)
        
        h_HSR = layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu"), name="HSRcontext")(context_mat_HSR)
        h_HSR = layers.TimeDistributed(layers.Flatten())(h_HSR)
        
        h_LL = layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu"), name="LLcontext")(context_mat_LL)
        h_LL = layers.TimeDistributed(layers.Flatten())(h_LL)
        
        h_LR = layers.TimeDistributed(layers.Conv1D(filters=64, kernel_size=3, strides=1, activation="relu"), name="LRcontext")(context_mat_LR)
        h_LR = layers.TimeDistributed(layers.Flatten())(h_LR)
        
        h_ML = layers.TimeDistributed(layers.Dense(1024, activation="relu"), name="MLcontext")(context_mat_ML)
        h_ML = layers.TimeDistributed(layers.Flatten())(h_ML)
        
        h_MR = layers.TimeDistributed(layers.Dense(1024, activation="relu"), name="MRcontext")(context_mat_MR)
        h_MR = layers.TimeDistributed(layers.Flatten())(h_MR)
        
        concat = layers.Concatenate(axis=2)([h_HSL,h_HSR,h_LL,h_LR,h_ML,h_MR])
#         h = layers.TimeDistributed(layers.Dense(512, activation="relu"))(concat)
#         h = layers.TimeDistributed(layers.Dense(256, activation="relu"))(h)
#         h = layers.TimeDistributed(layers.Dense(128, activation="relu"))(h)
        
#         h = layers.LSTM(128, return_sequences=True)(h)
        h = layers.LSTM(128, return_sequences=True)(concat)
#         h = layers.LSTM(128, return_sequences=True)(h)
        h = layers.TimeDistributed(layers.Dense(128, activation="relu"))(h)
        h = layers.TimeDistributed(layers.Dense(num_words+1, activation="softmax"), name="Outputlayer")(h)
        output = CTCLayer(name="ctc-loss")(labels.to_tensor(), h.to_tensor())

        model = keras.models.Model(
            inputs=[input_HSL, input_HSR, input_LL, input_LR, input_ML, input_MR, labels], outputs=output, name="test_SLT"
        )
        opt = keras.optimizers.Adam()
        
        # Compile the model and return
        model.compile(optimizer=opt)
        return model    
    
        # Handshape Attention block
model = build_model()
model.summary()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_dataset, batch_size=2, epochs=20, callbacks=tensorboard_callback)

Model: "test_SLT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
HSL (InputLayer)                [(None, None, 25, 10 0                                            
__________________________________________________________________________________________________
HSR (InputLayer)                [(None, None, 25, 10 0                                            
__________________________________________________________________________________________________
LL (InputLayer)                 [(None, None, 25, 12 0                                            
__________________________________________________________________________________________________
LR (InputLayer)                 [(None, None, 25, 12 0                                            
___________________________________________________________________________________________

C:\Users\minhyuklee\Anaconda3\envs\research\lib\site-packages\tensorflow\python\framework\indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/test_SLT/input.to_tensor_1/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/test_SLT/input.to_tensor_1/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 26), dtype=float32), dense_shape=Tensor("gradient_tape/test_SLT/input.to_tensor_1/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


90/90 [==============================] - 35s 198ms/step - loss: 15.9589
Epoch 2/20
90/90 [==============================] - 14s 154ms/step - loss: 15.1786
Epoch 3/20
90/90 [==============================] - 13s 141ms/step - loss: 14.3127
Epoch 4/20
90/90 [==============================] - 13s 142ms/step - loss: 12.3833
Epoch 5/20
90/90 [==============================] - 13s 144ms/step - loss: 8.7428
Epoch 6/20
90/90 [==============================] - 13s 146ms/step - loss: 6.7072
Epoch 7/20
90/90 [==============================] - 13s 149ms/step - loss: 5.3509
Epoch 8/20
90/90 [==============================] - 13s 143ms/step - loss: 4.9768
Epoch 9/20
90/90 [==============================] - 13s 143ms/step - loss: 4.3190
Epoch 10/20
90/90 [==============================] - 13s 143ms/step - loss: 3.9835
Epoch 11/20
90/90 [==============================] - 13s 144ms/step - loss: 4.2114
Epoch 12/20
90/90 [==============================] - 13s 149ms/step - loss: 4.4210
Epoch 13/20
90/90 [=

In [5]:
"""
# Test dataset load
# Get list of all the sentences
HSL_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\HSL\\testset"
HSR_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\HSR\\testset"
LL_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\LL\\testset"
LR_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\LR\\testset"
ML_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\ML\\testset"
MR_test_path = r"D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\DATA\\feature\\MR\\testset"

HSL_test_all_files = glob(HSL_test_path+"\\*.csv")
HSR_test_all_files = glob(HSR_test_path+"\\*.csv")
LL_test_all_files = glob(LL_test_path+"\\*.csv")
LR_test_all_files = glob(LR_test_path+"\\*.csv")
ML_test_all_files = glob(ML_test_path+"\\*.csv")
MR_test_all_files = glob(MR_test_path+"\\*.csv")

HSL_test_sentences = cubic_sentences(HSL_test_all_files)
HSR_test_sentences = cubic_sentences(HSR_test_all_files)
LL_test_sentences = cubic_sentences(LL_test_all_files)
LR_test_sentences = cubic_sentences(LR_test_all_files)
ML_test_sentences = cubic_sentences(ML_test_all_files)
MR_test_sentences = cubic_sentences(MR_test_all_files)

stc_seq = [[0,1,2],[3,4,5],[6,7,2],[8,5,9],[10,8,11],[4,2,9],[12,13,8,5],
          [0,14,15,16],[0,13,17,18],[19,13,1,20],[3,13,1,5],[15,8,6,21],
          [15,1,22,5],[10,7,0,18],[11,14,23,24,20],[0,7,23,22,5],[15,7,23,24,21],
          [10,8,23,24,21]] # 0부터 인덱싱
num_words = 25
labels = []
for filename in HSL_all_files:
    temp = filename.split(os.path.sep)[-1].split(".csv")[0]
    temp = temp.split('-')[1]
    temp = temp[-1]
    labels.append(np.array(stc_seq[int(temp)-1]))
labels_test = tf.ragged.constant(labels)

HSL_test_dataset = tf.ragged.constant(HSL_test_sentences, inner_shape=(window_size,10))
HSR_test_dataset = tf.ragged.constant(HSR_test_sentences, inner_shape=(window_size,10))
LL_test_dataset = tf.ragged.constant(LL_test_sentences, inner_shape=(window_size,12))
LR_test_dataset = tf.ragged.constant(LR_test_sentences, inner_shape=(window_size,12))
ML_test_dataset = tf.ragged.constant(ML_test_sentences, inner_shape=(1,window_size+3))
MR_test_dataset = tf.ragged.constant(MR_test_sentences, inner_shape=(1,window_size+3))

indices = tf.range(start=0, limit=len(HSL_test_sentences))
shuffled_indices = tf.random.shuffle(indices)

HSL_test_dataset = tf.gather(HSL_test_dataset, shuffled_indices)
HSR_test_dataset = tf.gather(HSR_test_dataset, shuffled_indices)
LL_test_dataset = tf.gather(LL_test_dataset, shuffled_indices)
LR_test_dataset = tf.gather(LR_test_dataset, shuffled_indices)
ML_test_dataset = tf.gather(ML_test_dataset, shuffled_indices)
MR_test_dataset = tf.gather(MR_test_dataset, shuffled_indices)
labels_test = tf.gather(labels_test, shuffled_indices)

test_dataset = encode_dict(HSL_test_dataset, HSR_test_dataset, LL_test_dataset, LR_test_dataset, ML_test_dataset, MR_test_dataset, labels_test)
"""



prediction_model = keras.models.Model(
    inputs=[model.get_layer(name="HSL").input, model.get_layer(name="HSR").input, model.get_layer(name="LL").input, 
            model.get_layer(name="LR").input, model.get_layer(name="ML").input, model.get_layer(name="MR").input], 
    outputs=model.get_layer(name="Outputlayer").output

)
# prediction_model.summary()

def decode_batch_predictions(pred):
    a = pred.numpy()
    input_len = []
    for i in range(len(a)):
        input_len = np.append(input_len, a[i].shape[0])
#     print(input_len)
    
    
#     input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred.to_tensor(), input_length=input_len, greedy=True)[0][0][:, :5]
    
    return results

save_dir = "D:\\MinHyuk\\Hand Sign Recognition\\2021_SLT_prj\\Analysis"


batch_HSL = train_dataset.get("HSL")
batch_HSR = train_dataset.get("HSR")
batch_LL = train_dataset.get("LL")
batch_LR = train_dataset.get("LR")
batch_ML = train_dataset.get("ML")
batch_MR = train_dataset.get("MR")
batch_labels = train_dataset.get("sentence")


"""
batch_HSL = test_dataset.get("HSL")
batch_HSR = test_dataset.get("HSR")
batch_LL = test_dataset.get("LL")
batch_LR = test_dataset.get("LR")
batch_ML = test_dataset.get("ML")
batch_MR = test_dataset.get("MR")
batch_labels = test_dataset.get("sentence")
"""

preds = prediction_model.predict([batch_HSL, batch_HSR, batch_LL, batch_LR, batch_ML, batch_MR])
pred_sentences = decode_batch_predictions(preds)
pred_sentences = pred_sentences.numpy()

for i in range(preds.shape[0]):
    dataframe1 = pd.DataFrame(preds[i].numpy())
    dataframe1.to_csv(save_dir+"\\prediction"+str(i)+".csv", header=False, index=False)
    
    dataframe2 = pd.DataFrame(pred_sentences[i])
    dataframe2.to_csv(save_dir+"\\pred_decoded"+str(i)+".csv", header=False, index=False)
    
    dataframe3 = pd.DataFrame(batch_labels[i].numpy())
    dataframe3.to_csv(save_dir+"\\true"+str(i)+".csv", header=False, index=False)

print('Prediction')
print(pred_sentences)

orig_labels = []
for label in batch_labels:
    label = label.numpy()
    orig_labels.append(label)

print('Ground truth')
print(orig_labels)
#    for i in range(len(pred_gestures)):
#        plt.figure(figsize=(20,5))
#        plt.subplot(3,2,i+1)
#        plt.plot(batch_sentences[i,:,:].numpy())
#        title = f"Prediction: {pred_gestures[i]}"
#        plt.title(title)

#print(pred_gestures)
#plt.show()

C:\Users\minhyuklee\Anaconda3\envs\research\lib\site-packages\tensorflow\python\ops\ragged\ragged_tensor.py:2063: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(rows)


Prediction
[[ 0  1  2 -1 -1]
 [ 4  2 -1 -1 -1]
 [ 0  7 23 22  5]
 [10  8 11 -1 -1]
 [ 4  2 -1 -1 -1]
 [ 0  7 23 22  5]
 [ 4  2 -1 -1 -1]
 [ 8  5  9 -1 -1]
 [15  1 22  5 -1]
 [15  8  6 21 -1]
 [10  8 23 24 21]
 [ 3  4  5 -1 -1]
 [ 6  7  2 -1 -1]
 [ 4  2  9 -1 -1]
 [ 3 13  1  5 -1]
 [15  8  6 21 -1]
 [ 8  5  9 -1 -1]
 [10  7  0 18 -1]
 [15  1 22  5 -1]
 [12 13  8  5 -1]
 [ 0 14 15 16 -1]
 [10  8 11 -1 -1]
 [ 6  7  2 -1 -1]
 [ 0  7 23 22  5]
 [ 0  1  2 -1 -1]
 [10  7  0 18 -1]
 [11 14 23 24 20]
 [11 14 23 24 20]
 [10  8 23 24 21]
 [15  1 22  5 -1]
 [11 14 23 24 20]
 [ 0  1  2 -1 -1]
 [11 14 23 24 20]
 [10  8 11 -1 -1]
 [10  7  0 18 -1]
 [19 13  1 20 -1]
 [ 8  5 -1 -1 -1]
 [ 3 13  1  5 -1]
 [15  8  6 21 -1]
 [15  1 22  5 -1]
 [ 0 13 17 18 -1]
 [10  8 11 -1 -1]
 [12 13  8  5 -1]
 [10  8 11 -1 -1]
 [12 13  8  5 -1]
 [12 13  8  5 -1]
 [ 3  4  5 -1 -1]
 [ 4  2 -1 -1 -1]
 [ 0 14 15 16 -1]
 [ 0 14 15 16 -1]
 [19 13  1 20 -1]
 [ 6  7  2 -1 -1]
 [15  8  6 21 -1]
 [ 3 13  1  5 -1]
 [ 3 13  1  5 -1]

In [ ]:
a = tf.constant([[[1,2,3],[4,5,6]], [[4,5,6],[7,8,9]]])
b = tf.constant([[1,2],[3,4],[5,6]])
c = tf.matmul(a,b)

In [ ]:
print(c)